In [4]:
import sys
sys.path.insert(0, '/home/xilinx/git/prjxray/tools/')

from ipywidgets import widgets
from IPython.display import display
from IPython.display import clear_output
from ipywidgets import Button, Layout, VBox, HBox

import os
import subprocess
from pynq.pl import Bitstream

#temp

'''
Create Verilog
This section takes care of creating a verilog file on the fly from user imput
'''
# ________________________________________________________________________________________________________________________
# Submit Verilog Section
# Text area object where user will type body of rp_top verilog
hdl_template = 'module rp_top(input clk, input [3:0]btn, input [9:0]sw, output [3:0]led, output [5:0]rgb_led, output [2:0]anode, output [6:0]segment, output dp, output txd);\n\n\n\nendmodule'

HDLbody = widgets.Textarea(value=hdl_template, layout=Layout(height='360px', width='825px'))

#Button for submitting verilog
submitHDL = widgets.Button(description ="Save Verilog") #button for submitting verilog HDL

#global variable for current module
current = 'notebook'

selectTitle = widgets.HTML(
    value ="<pre>     Select a Demo Module</pre>"
)
entireFlow = widgets.Button(description = "Run Entire Flow")
synthesize = widgets.Button(description ="Synthesize") #button for submitting verilog HDL) #button for submitting verilog HDL
implement = widgets.Button(description ="Implement") #button for submitting verilog HDL
generateBit = widgets.Button(description ="Generate Bitstream") #button for submitting verilog HDL
downloadBit = widgets.Button(description ="Program Bitstream") 

extraSpace = widgets.HTML(
    value ="<pre>        </pre>"
)

startChoice = widgets.ToggleButtons(
    options = ['Create Verilog', 'Use Demo Module'],
    description = 'Select: ',
    disabled = False,
    button_style = '',
    tooltips = ['Description of fast'],
    value = 'Create Verilog'
)

selectModule = widgets.Dropdown(
    options=['regfile', 'decoder', 'stopwatch', 'timergame', 'transmitter'],
    value='regfile',
    description='Select',
)

#create a HBox to hold entire layout for writeV tab or pickDemo tab
buttons_layout = VBox([entireFlow, synthesize, implement, generateBit, downloadBit, submitHDL])
writeV_layout = HBox([buttons_layout, VBox([HDLbody])])
extra_layout =  HBox([extraSpace, startChoice])
writeV_wrapper = VBox([extra_layout, writeV_layout])
pickDemo_layout = HBox([buttons_layout, VBox([selectTitle, selectModule])]) 
pickDemo_wrapper = VBox([extra_layout, pickDemo_layout])

def handle_submit_verilog(sender):
    global current
    current = 'notebook'
    v_file = open("/home/xilinx/cad_flow/reconfig_modules/notebook/src/notebook.v", "w+")
    v_file.write(HDLbody.value)
    print("\nVerilog file created\n")

def handle_change(change):
    global current
    if change['type'] =='change' and change['name'] =='value':
        current = change['new']

selectModule.observe(handle_change)


# Helper methods
# __________________________________________________________________________________________________________________________

# Method which will synthesize, implement, generate bitstream, and download instead of individually
def handle_entire_flow(sender):
    handle_synthesis()
    handle_implement()
    handle_generateBit()
    handle_downloadBit()

# # Methods for quickly redisplaying titles and buttons for two layouts
def display_write_verilog():
    global current
    current = 'notebook'
    display(writeV_wrapper)
    submitHDL.on_click(handle_submit_verilog)
    synthesize.on_click(handle_synthesis)
    implement.on_click(handle_implement)
    generateBit.on_click(handle_generateBit)
    downloadBit.on_click(handle_downloadBit)

def display_demo():
    global current
    current = 'regfile'
    display(pickDemo_wrapper)
    synthesize.on_click(handle_synthesis)
    implement.on_click(handle_implement)
    generateBit.on_click(handle_generateBit)
    downloadBit.on_click(handle_downloadBit)

# #__________________________________________________________________________________________________________________
# #Synthesis with a makefile
# #This section takes care of synthesizing verilog design

def handle_synthesis(sender):
    global current
    !cd /home/xilinx/cad_flow/reconfig_modules/{current}; pwd; make
    print("Finished Synthesis for " + current + "\n")

# #__________________________________________________________________________________________________________________
# #Implement Design using Java
# #This section takes care of Placing and Routing, etc of design

def handle_implement(sender):
    global current
    print("Start Implementation of " + current + "\n")
    !export RAPIDSMITH_PATH="/home/xilinx/git/RapidSmith2"; java -classpath "/home/xilinx/cad_flow/jars/*" -Xmx416m -Xss640k edu.byu.ece.partialreconfig.examples.MaverickDemo /home/xilinx/cad_flow/reconfig_modules/{current}/{current}.rscp
    print("Implementation done for " + current + "\n")

# #__________________________________________________________________________________________________________________
# #Generate Bitstream
# #This section takes care generating the partial bitstream

def handle_generateBit(sender):
    global current
    # Command to source the database shell script
    source_cmd = "source /home/xilinx/git/prjxray/database/pynq/settings.sh"

    # Command to run the FASM to FRM converter
    fasm2frm_cmd = "${XRAY_DIR}/tools/fasm2frame.py /home/xilinx/cad_flow/reconfig_modules/"+current+"/"+current+".fasm /home/xilinx/cad_flow/reconfig_modules/"+current+"/"+current+".frm"

    # Command to generate the partial bitstream
    partial_bitgen_cmd = "${XRAY_TOOLS_DIR}/partial_bitgen --part_name ${XRAY_PART} --part_file ${XRAY_PART_YAML} --bitstream_file /home/xilinx/cad_flow/reconfig_modules/default/default_rm.bit --frm_file /home/xilinx/cad_flow/reconfig_modules/"+current+"/"+current+".frm --output_file /home/xilinx/cad_flow/reconfig_modules/"+current+"/"+current+"_rm.bit"

    # Execute the commands
    !{source_cmd}; {fasm2frm_cmd}; {partial_bitgen_cmd}
    
    print("Generated bitstream for " + current + "\n")


generateBit.on_click(handle_generateBit)

# #__________________________________________________________________________________________________________________
# #Download Bitstream
# #This section takes care of downloading the partial bitstream to the board

def handle_downloadBit(sender):
    global current
    Bitstream("/home/xilinx/cad_flow/reconfig_modules/"+current+"/"+current+"_rm.bit", partial = True).download()
    print("Programmed bitstream for " + current + "\n")

downloadBit.on_click(handle_downloadBit)

#_______________________________________________________________________________________________
#Make button for running entire flow
entireFlow.on_click(handle_entire_flow)
# Display the write_verilog option on initial startup
#display_write_verilog()
display_demo()

# If you toggle the switch for creating verilog vs selecting module then clear output
# Then display all buttons, etc
def toggle_on_click(change):
    if change['type'] == 'change':
        choice = change['new']
    else:
        choice = change['old']
    clear_output()

    if choice == 'Create Verilog':
        display_write_verilog() # you can either write your own verilog

    else:
        if choice == 'Use Demo Module':
            display_demo()  # or you can choose a demo module
            
startChoice.observe(toggle_on_click, 'value')

  

VBox(children=(HBox(children=(HTML(value='<pre>        </pre>'), ToggleButtons(description='Select: ', options=('Create Verilog', 'Use Demo Module'), tooltips=['Description of fast'], value='Create Verilog'))), HBox(children=(VBox(children=(Button(description='Run Entire Flow', style=ButtonStyle()), Button(description='Synthesize', style=ButtonStyle()), Button(description='Implement', style=ButtonStyle()), Button(description='Generate Bitstream', style=ButtonStyle()), Button(description='Program Bitstream', style=ButtonStyle()), Button(description='Save Verilog', style=ButtonStyle()))), VBox(children=(Textarea(value='module rp_top(input clk, input [3:0]btn, input [9:0]sw, output [3:0]led, output [5:0]rgb_led, output [2:0]anode, output [6:0]segment, output dp, output txd);\n\n\n\nendmodule', layout=Layout(height='360px', width='825px')),))))))

/home/xilinx/cad_flow/reconfig_modules/notebook
Synthesizing with Yosys...
